### Statistics Development Notebook


In [1]:
%load_ext autoreload
%autoreload 2

import glob
import os
from copy import deepcopy


import liftout
import pandas as pd
import plotly.express as px
import streamlit as st
from autoscript_sdb_microscope_client.structures import AdornedImage
from liftout.structures import (AutoLiftoutState, Lamella, Sample,
                                load_experiment)
from liftout.tools.data import (AutoLiftoutStatistics,
                                calculate_statistics_dataframe,
                                create_history_dataframe)
pd.set_option('display.max_rows', None)
# EXPERIMENT_PATH = r"C:\Users\Admin\Github\autoliftout\liftout\log\dry-run-01-2022-11-09.01-30-41PM"
# stats = calculate_statistics_dataframe(EXPERIMENT_PATH)


from liftout.config import config

stats_path = os.path.join(config.LOG_DATA_PATH, "stats")

os.makedirs(stats_path, exist_ok=True)


#### Aggregated Statistics

In [13]:
# get all the directories starting with dm-E
EXPERIMENT_PATH = r"C:\Users\Admin\Github\autoliftout\liftout\log"
experiment_paths1 = glob.glob(os.path.join(EXPERIMENT_PATH, "dm-E1*"))
experiment_paths2 = glob.glob(os.path.join(EXPERIMENT_PATH, "dm-E2*"))

from pprint import pprint

experiment_paths =  experiment_paths1 + experiment_paths2

pprint(experiment_paths)

['C:\\Users\\Admin\\Github\\autoliftout\\liftout\\log\\dm-E10-04-16Nov22-2022-12-02.10-28-08AM',
 'C:\\Users\\Admin\\Github\\autoliftout\\liftout\\log\\dm-E10-16Nov22-2022-12-01.10-25-29AM',
 'C:\\Users\\Admin\\Github\\autoliftout\\liftout\\log\\dm-E10-16Nov22-2022-12-06.11-07-08AM',
 'C:\\Users\\Admin\\Github\\autoliftout\\liftout\\log\\dm-E10-2022-11-29.11-53-16AM',
 'C:\\Users\\Admin\\Github\\autoliftout\\liftout\\log\\dm-E10-2022-11-30.10-29-04AM',
 'C:\\Users\\Admin\\Github\\autoliftout\\liftout\\log\\Dm-E11-3Nov22-2022-11-14.01-20-47PM',
 'C:\\Users\\Admin\\Github\\autoliftout\\liftout\\log\\dm-E11-Nov03-2022-11-17.10-16-36AM',
 'C:\\Users\\Admin\\Github\\autoliftout\\liftout\\log\\dm-E11-Nov03-2022-11-22.10-17-06AM',
 'C:\\Users\\Admin\\Github\\autoliftout\\liftout\\log\\dm-E2-01-16Nov22-2022-12-08.10-44-14AM',
 'C:\\Users\\Admin\\Github\\autoliftout\\liftout\\log\\dm-E2-21Oct22-01-2022-12-09.10-51-59AM',
 'C:\\Users\\Admin\\Github\\autoliftout\\liftout\\log\\dm-E2-21Oct22-03-20

In [27]:
# EXPERIMENT_PATHS = [
#     r"C:\Users\Admin\Github\autoliftout\liftout\log\Dm-E11-3Nov22-2022-11-14.01-20-47PM", 
#     r"C:\Users\Admin\Github\autoliftout\liftout\log\Dm_E11_3Nov2022_-2022-11-15.11-50-17AM",
#     r"C:\Users\Admin\Github\autoliftout\liftout\log\dm-E11-Nov03-2022-11-17.10-16-36AM"

#     ]

# TODO: add experiment name / date to each dataframe? 
# add date to sample...

df_ml, df_gamma, df_history, df_sample, df_clicks = None, None, None, None, None

for path in experiment_paths:
    
    if "dm-E2-01" in path:
        continue

    stats = calculate_statistics_dataframe(path)
    print(stats.name, stats.date)
    # display(stats.sample)
    # display(stats.ml)

    # concat stats.clicks in a single dataframe  
    if df_clicks is None:
        df_clicks = stats.click
    else:
        df_clicks = pd.concat([df_clicks, stats.click])

    # concat stats.ml in a single dataframe  
    if df_ml is None:
        df_ml = stats.ml
    else:
        df_ml = pd.concat([df_ml, stats.ml])

    # concat stats.gamma in a single dataframe
    if df_gamma is None:
        df_gamma = stats.gamma
    else:
        df_gamma = pd.concat([df_gamma, stats.gamma])


    # concat stats.history in a single dataframe
    if df_history is None:
        df_history = stats.history
    else:
        df_history = pd.concat([df_history, stats.history])

    # concat stats.sample in a single dataframe
    if df_sample is None:
        df_sample = stats.sample
    else:
        df_sample = pd.concat([df_sample, stats.sample])
        
        

--------------------------------------------------------------------------------
dm-E10-04-16Nov22-2022-12-02.10-28-08AM 2022-12-02 10:28:08
--------------------------------------------------------------------------------
dm-E10-16Nov22-2022-12-01.10-25-29AM 2022-12-01 10:25:29
--------------------------------------------------------------------------------
dm-E10-16Nov22-2022-12-06.11-07-08AM 2022-12-06 11:07:08
--------------------------------------------------------------------------------
dm-E10-2022-11-29.11-53-16AM 2022-11-29 11:53:16
--------------------------------------------------------------------------------
dm-E10-2022-11-30.10-29-04AM 2022-11-30 10:29:04
--------------------------------------------------------------------------------
Dm-E11-3Nov22-2022-11-14.01-20-47PM 2022-11-14 13:20:47
--------------------------------------------------------------------------------
dm-E11-Nov03-2022-11-17.10-16-36AM 2022-11-17 10:16:36
--------------------------------------------------

In [28]:
# group df_click by date and stage

df_clicks["date"] = pd.to_datetime(df_clicks["date"])
df_clicks["stage"] = df_clicks["stage"].astype("category")
df_click_group = df_clicks.groupby(by=["date", "stage"]).count()
df_click_group = df_click_group.reset_index()
df_click_group["date"] = df_click_group["date"].dt.strftime("%d-%m-%Y")

# add a count column
df_click_group["n_clicks"] = df_click_group["x"]

# drop all columns except for date, stage and n_clicks
df_click_group = df_click_group[["date", "stage", "n_clicks"]]

# display(df_click_group)

# group df_history by date and stage

df_history["date"] = pd.to_datetime(df_history["date"])
df_history["stage"] = df_history["stage"].astype("category")
df_history_group = df_history.groupby(by=["date", "stage"]).count()
df_history_group = df_history_group.reset_index()
df_history_group["date"] = df_history_group["date"].dt.strftime("%d-%m-%Y")

# add a count column
df_history_group["n_stages"] = df_history_group["duration"]



# drop all columns except for date, stage and n_stages
df_history_group = df_history_group[["date", "stage", "n_stages"]]
# display(df_history_group)

# merge df_click_group and df_history_group
df_click_history = pd.merge(df_click_group, df_history_group, on=["date", "stage"])

# add a column with the ratio of clicks/stages
df_click_history["clicks/stages"] = df_click_history["n_clicks"] / df_click_history["n_stages"]

# fillna 0
df_click_history = df_click_history.fillna(0)

# plot the time series with a title
fig = px.line(df_click_history, x="date", y="clicks/stages", color="stage")
fig.update_layout(title="Clicks per stage")
fig.show()

# fig.write_image(os.path.join(stats_path, "clicks_per_stage.png"))



In [25]:
# group df_click by date, stage and  source

df_clicks["date"] = pd.to_datetime(df_clicks["date"])
df_clicks["stage"] = df_clicks["stage"].astype("category")
df_clicks["source"] = df_clicks["source"].astype("category")
df_click_group = df_clicks.groupby(by=["date", "stage", "source"]).count()
df_click_group = df_click_group.reset_index()
df_click_group["date"] = df_click_group["date"].dt.strftime("%d-%m-%Y")

# add a count column
df_click_group["n_clicks"] = df_click_group["x"]

# drop all columns except for date, stage and n_clicks
df_click_group = df_click_group[["date", "stage", "source", "n_clicks"]]
# display(df_click_group)

# drop rows with n_clicks = 0
df_click_group = df_click_group[df_click_group["n_clicks"] > 0]

# sort by date
# df_click_group = df_click_group.sort_values(by="date", ascending=True)

# display(df_click_group)
# plot the time series with a title
fig = px.line(df_click_group, x="date", y="n_clicks", color="stage", line_dash="source")
fig.update_layout(title="Clicks per stage, source")
fig.show()


In [26]:
# group df_click by date, source
df_clicks["date"] = pd.to_datetime(df_clicks["date"])
df_clicks["source"] = df_clicks["source"].astype("category")
df_click_group = df_clicks.groupby(by=["date", "source"]).count()
df_click_group = df_click_group.reset_index()
df_click_group["date"] = df_click_group["date"].dt.strftime("%d-%m-%Y")

# add a count column
df_click_group["n_clicks"] = df_click_group["x"]

# drop all columns except for date, stage and n_clicks
df_click_group = df_click_group[["date", "source", "n_clicks"]]

# plot the time series with a title
fig = px.line(df_click_group, x="date", y="n_clicks", color="source")
fig.update_layout(title="Clicks per source")
fig.show()



In [18]:
# calculate the mean click size x and y for each date, stage
df_clicks["date"] = pd.to_datetime(df_clicks["date"])
df_clicks["stage"] = df_clicks["stage"].astype("category")
df_clicks_group = df_clicks.groupby(by=["date", "type"]).mean()
df_clicks_group = df_clicks_group.reset_index()

# fillna 0 for columns x and y
df_clicks_group[["x", "y"]] = df_clicks_group[["x", "y"]].fillna(0)

# plot the time series with a title
fig = px.line(df_clicks_group, x="date", y=["x", "y"], line_dash="type")
fig.update_layout(title="Mean click size")
fig.show()

In [19]:
# group df_ml by date, stage, feature, and correct
df_ml["date"] = pd.to_datetime(df_ml["date"])
df_ml["stage"] = df_ml["stage"].astype("category")
df_ml["feature"] = df_ml["feature"].astype("category")
df_ml_group = df_ml.groupby(by=["date", "stage", "feature", "correct"]).count()
df_ml_group = df_ml_group.reset_index()
df_ml_group["date"] = df_ml_group["date"].dt.strftime("%d-%m-%Y")

# convert correct to bool
df_ml_group["correct"] = df_ml_group["correct"].apply(lambda x: True if x == "True" else False)

# add a count column 
df_ml_group["count"] = df_ml_group["name"]

# drop all columns except date, stage, feature, correct, count
df_ml_group = df_ml_group[["date", "stage", "feature", "correct", "count"]]

# calculate the percentage of correct predictions
df_ml_group_correct = df_ml_group[df_ml_group["correct"] == True]
df_ml_group_incorrect = df_ml_group[df_ml_group["correct"] == False]

df_ml_group_correct = df_ml_group_correct.rename(columns={"count": "correct_count"})
df_ml_group_incorrect = df_ml_group_incorrect.rename(columns={"count": "incorrect_count"})
df_ml_group_correct = df_ml_group_correct[["date", "stage", "feature", "correct_count"]]
df_ml_group_incorrect = df_ml_group_incorrect[["date", "stage", "feature", "incorrect_count"]]
df_ml_group = pd.merge(df_ml_group_correct, df_ml_group_incorrect, on=["date", "stage", "feature"])
df_ml_group["total"] = df_ml_group["correct_count"] + df_ml_group["incorrect_count"]
df_ml_group["percentage_correct"] = df_ml_group["correct_count"] / df_ml_group["total"] * 100

# fillna with 0
df_ml_group["percentage_correct"] = df_ml_group["percentage_correct"].fillna(0)

# drop rows with total = 0
df_ml_group = df_ml_group[df_ml_group["total"] != 0]

# plot a time series of total and percentage correct per feature with different line styles for stage
fig = px.line(df_ml_group, x="date", y="total", color="stage", line_dash="feature")
# title
fig.update_layout(title="Total number of predictions per feature")
fig.show()

# add a title
fig = px.line(df_ml_group, x="date", y="percentage_correct", color="stage", line_dash="feature")
fig.update_layout(title="Percentage of correct predictions per feature")
fig.show()


In [20]:
# group df_history by date and stage
df_history["date"] = pd.to_datetime(df_history["date"])
df_history["stage"] = df_history["stage"].astype("category")
df_history_group = df_history.groupby(by=["date", "stage"]).mean()
df_history_group = df_history_group.reset_index()
df_history_group["date"] = df_history_group["date"].dt.strftime("%d-%m-%Y")

# drop all columns except date, stage, duration
df_history_group = df_history_group[["date", "stage", "duration"]]
df_history_group["duration"] = df_history_group["duration"].astype("float")

# fillna 0
df_history_group["duration"] = df_history_group["duration"].fillna(0)

# plot a time series of duration per stage
fig = px.line(df_history_group, x="date", y="duration", color="stage")
fig.update_layout(title="Average duration per stage")
fig.show()


In [21]:
# group df_history by date and stage
df_history["date"] = pd.to_datetime(df_history["date"])
df_history["stage"] = df_history["stage"].astype("category")
df_history_group = df_history.groupby(by=["date", "stage"]).count()
df_history_group = df_history_group.reset_index()
df_history_group["date"] = df_history_group["date"].dt.strftime("%d-%m-%Y")

# drop all columns except date, stage, duration

df_history_group = df_history_group[["date", "stage", "duration"]]

# fillna 0
df_history_group["duration"] = df_history_group["duration"].fillna(0)

# get how many stage = MillTrench for each date
df_history_group_milltrench = df_history_group[df_history_group["stage"] == "MillTrench"]

# get how many stage = Liftout for each date
df_history_group_liftout = df_history_group[df_history_group["stage"] == "Liftout"]

# get how many stage = Landing for each date
df_history_group_landing = df_history_group[df_history_group["stage"] == "Landing"]

# get how many stage = Finished for each date
df_history_group_finished = df_history_group[df_history_group["stage"] == "Finished"]

# merge all the  dataframes
df_history_group = pd.merge(df_history_group_milltrench, df_history_group_liftout, on="date")

# rename columns
df_history_group = df_history_group.rename(columns={"duration_x": "milltrench", "duration_y": "liftout"})

df_history_group = pd.merge(df_history_group, df_history_group_landing, on="date")
df_history_group = pd.merge(df_history_group, df_history_group_finished, on="date")

# rename columns
df_history_group = df_history_group.rename(columns={"duration_x": "landing", "duration_y": "finished"})

# drop
df_history_group = df_history_group[["date", "milltrench", "liftout", "landing", "finished"]]


# display(df_history_group)

# calculate completion rate milltrench / milltrench
df_history_group["milltrench_completetion_rate"] = df_history_group["milltrench"] / df_history_group["milltrench"] * 100

# calculate completion rate finished / milltrench
df_history_group["finish_completion_rate"] = df_history_group["finished"] / df_history_group["milltrench"] * 100

# calculate completion rate landing / milltrench
df_history_group["landing_completion_rate"] = df_history_group["landing"] / df_history_group["milltrench"] * 100

# calculate completion rate liftout / milltrench
df_history_group["liftout_completion_rate"] = df_history_group["liftout"] / df_history_group["milltrench"] * 100

# plot a time series of completion rate with dashed lines
fig = px.line(df_history_group, x="date", y=["milltrench_completetion_rate", 
    "liftout_completion_rate", 
    "landing_completion_rate", "finish_completion_rate"])
fig.update_layout(title="Completion rate")
fig.show()


C:\Users\Admin\AppData\Local\Temp\ipykernel_8688\1857523922.py:34: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'stage_x'} in the result is deprecated and will raise a MergeError in a future version.



In [22]:
# group df_gamma by date, and beam_type
df_gamma["date"] = pd.to_datetime(df_gamma["date"])
df_gamma["beam_type"] = df_gamma["beam_type"].astype("category")
df_gamma_group = df_gamma.groupby(by=["date", "beam_type"]).mean()
df_gamma_group = df_gamma_group.reset_index()
df_gamma_group["date"] = df_gamma_group["date"].dt.strftime("%d-%m-%Y")

# drop all columns except date, beam_type, gamma
df_gamma_group = df_gamma_group[["date", "beam_type", "gamma"]]
df_gamma_group["gamma"] = df_gamma_group["gamma"].astype("float")

# fillna 0
df_gamma_group["gamma"] = df_gamma_group["gamma"].fillna(0)

# plot a time series of gamma per beam_type
fig = px.line(df_gamma_group, x="date", y="gamma", color="beam_type")
fig.update_layout(title="Average gamma per beam type")
fig.show()
